In [7]:
import json
import os
import shutil
import random
import math
import numpy as np

from glob import glob
from PIL import Image, ImageDraw
from tqdm import tqdm

In [8]:
def oversampling_on_crack(src_path, dest_path):
    for name in tqdm(glob(os.path.join(src_path, '*.json'))):
        with open(name) as fo:
            ann = json.load(fo)
        
        all_crack = []
        
        if len([shape for shape in ann['shapes'] if shape['label'] == 'repair']) != 0:
            continue    
        if len([shape for shape in ann['shapes'] if shape['label'] == 'crack']) < 1:
            continue
        
        crack = []
        for shape in ann['shapes']:
            # 存放crack标签的points   
            if shape['label'] == 'crack':
                crack_xy = [tuple(p) for p in shape['points']]
                crack.append(crack_xy)
                
        for each_crack in crack:
            # crack_points_4：crack外接框的四个顶点坐标
            crack_points_4 = crack_external_frame(each_crack)
            
            # 默认左上角为支撑点
            domination_point = crack_points_4[0]
    
            # 读取该json文件对应的bmp文件，得到其mask，再将mask截为crack外接框大小
            image = Image.open('{}.bmp'.format(name[:-5]))

            mask = Image.new('1', image.size)
            mask_draw = ImageDraw.Draw(mask)
            mask_draw.polygon(crack[0], fill=1)
            
            cut_box = (crack_points_4[0] + crack_points_4[1])

            cut_original_image = image.crop(cut_box)
            cut_mask_image = mask.crop(cut_box)
            
            # 归零
            left, top = crack_points_4[0]
            domination_point = (domination_point[0] - left, domination_point[1] - top)
            each_crack = [(x - left, y - top) for x, y in each_crack]

            all_crack.append((cut_original_image, cut_mask_image, domination_point, crack))

            image = Image.open('{}.bmp'.format(name[:-5]))
            images_ann = (image, mask, ann)

        copy_image(src_path, dest_path, images_ann, all_crack)

In [9]:
def crack_external_frame(points_crack):
    all_x = [x for x, y in points_crack]
    all_y = [y for x, y in points_crack]
    
    max_x = max(all_x)
    min_x = min(all_x)
    max_y = max(all_y)
    min_y = min(all_y)
    
    external_frame_points = ((min_x, min_y), 
                             (max_x, max_y))
    
    return external_frame_points # 中心点和外接框的四角

In [10]:
def copy_image(src_path, dest_path, images_ann, all_crack):
#     print(all_crack)
    im, ori_mask, ann = images_ann
    for pic_and_mask in all_crack:
        pic, \
        mask, \
        domination_point, \
        points_crack = pic_and_mask
        
        if ((pic.width ** 2 + pic.height ** 2) ** 1/2 ) < 200:
            continue
            
        time = 5
        while(time):
            rotate_angel = random.randrange(0,360)

            pic_processed = pic.rotate(rotate_angel, expand = True)
            mask_processed = mask.rotate(rotate_angel, expand = True)

#             print(range_max_x)
            paste_x = random.randrange(-(pic_processed.width // 2), im.width)
            paste_y = random.randrange(-(pic_processed.height // 2), im.height)

            key_point = (paste_x, paste_y)

#             print(key_point)
#             display(mask)
            im.paste(pic_processed, key_point, mask = mask_processed)
            ori_mask.paste(mask_processed, key_point, mask = mask_processed)
#             display(ori_mask)
            time -= 1

            # 保存图片和对应的json文件
        save_image_mask(src_path, dest_path, im, ori_mask, ann)

In [11]:
def save_image_mask(src_path, dest_path, image, mask, ann):
    image.save(os.path.join(dest_path, 'crack_f2/', ann['imagePath']))
    
    mask_draw = ImageDraw.Draw(mask)

    for shape in ann['shapes']:
        if shape['label'] != 'crack' or shape['shape_type'] != 'polygon':
            continue
        mask_draw.polygon([tuple(p) for p in shape['points']], fill=1)
    mask.save(os.path.join(dest_path, 'crack_f2/','{}.png'.format(ann['imagePath'][:-4] )))
    

In [12]:
oversampling_on_crack("C:/Users/JST/APD202004v2/train/", "C:/Users/JST/APD202004v2/")

100%|██████████████████████████████████████████████████████████████████████████████| 2769/2769 [03:52<00:00, 11.91it/s]
